<a href="https://colab.research.google.com/github/aguilin1/tda_ai_text_generation/blob/anns-edits/tda_ai_text_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import csv
import pandas as pd

data = pd.read_csv('/content/tda_ai_text_generation/research-abstracts-labeled-first-100.csv')
# Print first 4 rows to sanity check
data[:4]

,title,label,text,word_count
0,Study of coupling loss on bi-columnar BSCCO/Ag...,0,Coupling losses were studied in composite tape...,280
1,Study of coupling loss on bi-columnar BSCCO/Ag...,1,"In this study, we investigate the coupling los...",215
2,Weighted Solyanik estimates for the strong max...,0,Let $\mathsf M_{\mathsf S}$ denote the strong ...,332
3,Weighted Solyanik estimates for the strong max...,1,"In this paper, we investigate Weighted Solyani...",225
